# Import a new version/extension of CIDOC CRM base
------------------------

## Informations en français

Ce carnet prépare le fichier à importer dans OntoME, à partir d'un fichier dans le dossier "input".

Script original`import_crm.ipynb` écrit par @atterebf, forké et modifié par @MPica.

⟶ Attention aux cellules signalées avec des symboles ✗ dans le titre : elles demandent de saisir ou vérifier certaines informations manuellement. ⟵

#### Modifications apportées :
* utilisation de `rdflib` pour importer tout fichier dans un graphe et travailler directement à partir dudit graphe ;
* ajout de la gestion du nom de la version dans le script lui-même, afin de pouvoir utiliser ce script sur toutes les versions du CIDOC CRM, via les variables `current_version`, `being_imported`, `import_desc` et `import_version` ;
* import des numéros des espaces de nom disponibles dans OntoME depuis un fichier JSON externe (évolutif), dans `ontome_ns` ;
* vérification des propriétés disponibles dans le script en l'état afin de savoir si l'on doit le modifier avant utilisation ;
* création d'un dictionnaire évolutif dans la variable `ns`, des espaces de noms avec leurs préfixes.

#### Note :
Si un nœud renvoie une erreur "ne doit pas être NoneType", c'est probablement que j'attendais une URI en écrivant le script et que j'ai un nœud de type liste ou autre. Ajouter une vérification de type : `if type(domain) == rdflib.term.URIRef:`

----------------------

> ## English information

> *This notebook prepares the file to be imported in OntoME, from a file in the "input" folder.*

> *The original `import_crm.ipynb` script was written by @atterebf, then forked and modified by @MPica.*

> ⟶ *Attention: the sections marked with ✗ symbols in their title need some informations entered or checked manually.* ⟵

> #### Modifications made :
> * *use of `rdflib` to import any file in a graph and work directly from said graph;*
> * *name of version is added within the script itself, so that this script may be used on all versions and extensions of the CIDOC-CRM, thanks to the variables `current_version`, `being_imported`, `import_desc` and `import_version`*;
> * *the import of the namespace numbers available in OntoME from an external upgradeable Json file, into variable `ontome_ns`*;
> * *checking the properties available in the script as it is, so as to know if it needs to be modified before use*;
> * *creation of an upgradeable dictionary in the variable `ns`, with namespaces and their prefixes*.

> #### Note :
> *If a node returns a "should not be NoneType" error, it probably means that I was expecting a URI when I wrote the script, but the node actually is a list or something else. A check should be added, such as: `if type(domain) == rdflib.term.URIRef:`*

----------------------

TO-DO NOTE : CRM:NOTE along with RDFS:LABEL

## Imports

In [3]:
import lxml.etree as et
import lxml.html as ht
from datetime import datetime
import re
from itertools import chain
import json
import numpy as np
import rdflib

## ✗✗✗ Déclarations sur l'ontologie à transformer ✗✗✗

Ces variables doivent être redéfinies à chaque nouvelle version/extension.
> *Declarations on the ontology to transform. These variables must be redefined at each new extension/version.*

| Français | Variable | *English* |
| --------: | :--------: | :------- |
| contient le nom du fichier à traiter, sans l'extension de format. Cette variable servira à la fois pour parser le fichier d'origine, et comme identifiant. | **`current_version`** | *contains the name of the file to transform, without the format extension.* |
| contient le lien officiel pour l'espace de nom concerné par l'importation. | **`being_imported`** | *contains the official link for the imported namespace.* |
| servira à la définition du rdfs:standardLabel. Cette information n'étant généralement pas conservée ainsi dans le fichier d'origine, il est nécessaire d'aller la chercher manuellement. | **`import_desc`** | *will be used to define the rdfs:standardLabel. This information is usually not kept in the original file this way, therefore one must fetch it manually.* |
| (idem précédent). | **`import_version`** | *(same as previous)* |

Ex:
```python
current_version = "doremus"
being_imported = "http://data.doremus.org/ontology#"
import_desc = "DoReMus v0.2"
import_version = '0.2 (05/06/2017)'
```

In [4]:
current_version = "doremus"
being_imported = "http://data.doremus.org/ontology#"
import_desc = "DoReMus v0.2"
import_version = '0.2 (05/06/2017)'

## ✗✗✗ Déclarations d'espaces de nom ✗✗✗
À compléter si besoin.

EN: *namespace declarations, to be completed if need be.*

In [5]:
ns = {
    "xml" : "http://www.w3.org/XML/1998/namespace",
    "xsd" : "http://www.w3.org/2001/XMLSchema#",
    "rdf" : "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "rdfs" : "http://www.w3.org/2000/01/rdf-schema#",
    "owl" : "http://www.w3.org/2002/07/owl#",
    "skos" : "http://www.w3.org/2004/02/skos/core#",
    "vann" : "http://purl.org/vocab/vann/",
    "dc" : "http://purl.org/dc/elements/1.1/",
    "dct" : "http://purl.org/dc/terms/",
    "mus" : "http://data.doremus.org/ontology#",
    "crm" : "http://www.cidoc-crm.org/cidoc-crm/",
    "ecrm" : "http://erlangen-crm.org/current/",
    "frbroo" : "http://iflastandards.info/ns/fr/frbr/frbroo/",
    "efrbroo" : "http://erlangen-crm.org/efrbroo/"
}

## Déclarations de versions

Quand OntoME a plusieurs versions, il faut préciser ici manuellement laquelle sera utilisée pour l'import. Les versions disponibles sont indiquées en commentaire. Toutes les versions sont l'unique version dans OntoME sauf, pour l'instant, le CIDOC-CRM (7.1.1, 6.2 ou 5.0.4).
* Pour le CIDOC-CRM, la version spécifiée vaudra également pour eCRM.
* Pour FRBRoo, la version spécifiée vaudra également pour eFRBR.

#### English:

> *Version declaration.*
> *When OntoME has several versions, the one to use for the current import must be specified here. The available versions are indicated in a comment. Every version is the only one available in OntoME, except, for now, the CIDOC-CRM (7.1.1, 6.2 ou 5.0.4).*
> * *For the CIDOC-CRM, the specified version will also be this of eCRM.*
> * *For FRBRoo, the specified version will also be this of FRBR.*

In [6]:
specified_versions = {
    "http://www.cidoc-crm.org/cidoc-crm/" : "6.2",
    "https://ontome.net/ns/symogih" : "1.3",
    "http://www.cidoc-crm.org/cidoc-crm/CRMarchaeo/" : "1.4.1",
    "https://ontome.net/ns/hisarc-rdf" : "0.0.1",
    "https://ontome.net/ns/test-namespace" : "Version 6",
    "http://momaf-data.utu.fi/" : "0.1",
    "https://ontome.net/ns/gemenet" : "0.1",
    "http://erlangen-crm.org/current/" : "6.2"
}

## Import du dictionnaire externe

Récupérer les correspondances entre les URI et leurs numéros, pour les espaces de nom déjà présents sur OntoME, depuis un fichier Json externe. Ce fichier comporte un dictionnaire, avec les espaces de nom déjà présents sur OntoME, sous ce modèle:
```json
"http://www.cidoc-crm.org/cidoc-crm/": [
    {"version": "7.1.1", "id": 188},
    {"version": "5.0.4", "id": 187},
    {"version": "6.2", "id": 1}
]
```
Sont égalements présents des espaces de nom non présents dans OntoME en eux-mêmes, mais dont les éléments peuvent être alignés sur un autre qui est, lui, présent. Actuellement :
* eCRM -> CIDOC-CRM
* eFRBRoo -> FRBRoo

> #### English:

> *Importing external dictionary. Get the correspondances between the URIs and their numbers, for the namespaces already in OntoME, from an external Json file. This file includes a dictionary, with these namespaces, according to the model above.*

> *Some namespaces not included in OntoME are also present, when their elements may be aligned on another which is. As of now, they are:*
> * eCRM -> CIDOC-CRM
> * eFRBRoo -> FRBRoo

In [7]:
with open('references/list_ns_uri_220907.json') as jsontome:
    ontome_ns = json.load(jsontome)

## Initialisation du parseur
> *Initialisation of the parser.*

Documentation : https://lxml.de/validation.html#xmlschema

In [8]:
f = 'references/schemaImportXmlwithReferences.xml'
xmlschema_doc = et.parse(f)
xmlschema = et.XMLSchema(xmlschema_doc)
type(xmlschema)

lxml.etree.XMLSchema

------------------------
## ✗✗✗ Importation du document à traiter ✗✗✗

**Attention** : changer l'extension du fichier source dans la première cellule si besoin.

**Attention**, si la cellule **2** extrait des types autres que :
* owl:Class,
* owl:Ontology,
* owl:ObjectProperty,
* owl:DatatypeProperty,

penser à vérifier si ces informations sont nécessaires, auquel cas le script doit être complété pour pouvoir les traiter.

**Attention** également : les cellules **3** et **4** peuvent signaler la présence de propriétés non traitées par le script. Dans le cas contraire, elles ne donnent que le nombre des classes/propriétés trouvées et les cinq premières.

#### English:

> **Attention**: *change the extension of the source file in the first cell if needed.*

> **Attention**: *if cell **2** extracts types other than:*
> * owl:Class,
> * owl:Ontology,
> * owl:ObjectProperty,
> * owl:DatatypeProperty,

> *please check if these informations are necessary, in which case the script must be completed to be able to treat them.*

> **Attention**: *cells *3* and *4* may show properties not treated in the script. Otherwise, they only give the name of classes/properties found and the first 5.

In [9]:
file = f'input/{current_version}.ttl'

In [10]:
### Ouvrir et importer le fichier
### Open and import file.
try:
    g = rdflib.Graph()
    g.parse(file)
    
    # Préparer la déclaration des espaces de nom.
    # Prepare namespace declarations.
    ns_decl_list = []
    
    # Préparer les vérifications
    # Prepare checks.
    typed_decls = {}
    typelist = []
    for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), None)):
        # Ces éléments changeront de valeur lorsqu'ils seront traités pour output.
        # These elements will have different values when they're dealt with.
        typed_decls[s] = "no"
        typelist.append(o)
    print("Voici les objets possibles pour rdf:type :")
    print(np.unique(typelist))
    
except Exception as e:
    print('Error: ' + str(e))

Voici les objets possibles pour rdf:type :
['http://www.w3.org/2002/07/owl#AnnotationProperty'
 'http://www.w3.org/2002/07/owl#Class'
 'http://www.w3.org/2002/07/owl#DatatypeProperty'
 'http://www.w3.org/2002/07/owl#InverseFunctionalProperty'
 'http://www.w3.org/2002/07/owl#IrreflexiveProperty'
 'http://www.w3.org/2002/07/owl#NamedIndividual'
 'http://www.w3.org/2002/07/owl#ObjectProperty'
 'http://www.w3.org/2002/07/owl#Ontology'
 'http://www.w3.org/2002/07/owl#Restriction'
 'http://www.w3.org/2002/07/owl#SymmetricObjectProperty']


In [12]:
### Tester la récupération des 'classes',
# ainsi que les prédicats dont elles sont sujet.
### Test the recovery of 'classes',
# as well as the predicates of which they are subjects.

classes = []
cpredispo = [
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#type",
    "http://www.w3.org/2000/01/rdf-schema#comment",
    "http://www.w3.org/2000/01/rdf-schema#isDefinedBy",
    "http://www.w3.org/2000/01/rdf-schema#label",
    "http://www.w3.org/2000/01/rdf-schema#subClassOf",
    "http://www.w3.org/2002/07/owl#equivalentClass",
    "http://www.w3.org/2002/07/owl#unionOf"
]
cpredicates = []

# Récupérer les prédicats.
# Get the predicates.
for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#Class'))):
    classes.append(s)
    print(s)
    for so, po, oo in g.triples((s, None, None)):
        cpredicates.append(po)
        print(po)

# Output direct.
print(f"Il y a {len(classes)} classes. Voici les 5 premières :\n")
for c in classes[:5]:
    print(type(c), c)

# Ne crée un output que si des prédicats
# ne sont pas encore dans le script.
# Only prints something if some of the
# predicates are not yet in the script.
for p in np.unique(cpredicates):
    if p not in cpredispo:
        print("Cette propriété n'est pas encore gérée dans ce script et doit y être rajoutée :\n\t", p)

n9672fb80a1cc4d2a95e8dd70a01bf262b1
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2002/07/owl#unionOf
n9672fb80a1cc4d2a95e8dd70a01bf262b4
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2002/07/owl#unionOf
http://data.doremus.org/ontology#M10_Catalogue_Name
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://www.w3.org/2000/01/rdf-schema#isDefinedBy
http://www.w3.org/2000/01/rdf-schema#label
http://www.w3.org/2000/01/rdf-schema#label
http://data.doremus.org/ontology#M11_Catalogue_Number
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://www.w3.org/2000/01/rdf-schema#isDefinedBy
http://www.w3.org/2000/01/rdf-schema#label
http://www.w3.org/2000/01/rdf-schema#label
http://data.doremus.org/ontology#M12_Opus_Number
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://www.w3.org/2000/01/rdf-schema#subClassOf
http://www.w3.org/2000/01/rdf-schema#isDefi

In [15]:
### Tester la récupération des 'propriétés'.
# Même fonctionnement qu'au-dessus.
# À savoir : les owl:ObjectProperty et owl:DatatypeProperty
# sont traitées ici de concert.

### Test the recovery of 'properties'.
# It works the same way as the above cell.
# Good to know: owl:ObjectProperty and owl:DatatypeProperty
# are treated together here.


props = []
ppredispo = [
    "http://www.w3.org/1999/02/22-rdf-syntax-ns#type",
    "http://www.w3.org/2000/01/rdf-schema#comment",
    "http://www.w3.org/2000/01/rdf-schema#domain",
    "http://www.w3.org/2000/01/rdf-schema#isDefinedBy",
    "http://www.w3.org/2000/01/rdf-schema#label",
    "http://www.w3.org/2000/01/rdf-schema#range",
    "http://www.w3.org/2000/01/rdf-schema#subPropertyOf",
    "http://www.w3.org/2002/07/owl#inverseOf",
    "http://www.w3.org/2004/02/skos/core#scopeNote"
]
ppredicates = []

# Récupérer les prédicats.
# Get the predicates.
for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#ObjectProperty'))):
    props.append(s)
    for so, po, oo in g.triples((s, None, None)):
        ppredicates.append(po)
        
for s, p, o in g.triples((None, rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2002/07/owl#DatatypeProperty'))):
    props.append(s)
    for so, po, oo in g.triples((s, None, None)):
        ppredicates.append(po)

# Output direct.
print(f"Il y a {len(props)} propriétés. Voici les 5 premières :\n")
for p in props[:5]:
    print("\t", type(p), p)

# Ne crée un output que si des prédicats
# ne sont pas encore dans le script.
# Only prints something if some of the
# predicates are not yet in the script.
for p in np.unique(ppredicates):
    if p not in ppredispo:
        print("\nCette propriété n'est pas encore gérée dans ce script et doit y être rajoutée :\n\t", p)

Il y a 282 propriétés. Voici les 5 premières :

	 <class 'rdflib.term.URIRef'> http://data.doremus.org/ontology#CLU198i_should_be_container_of
	 <class 'rdflib.term.URIRef'> http://data.doremus.org/ontology#CLU197_should_have_binding
	 <class 'rdflib.term.URIRef'> http://data.doremus.org/ontology#CLU197i_should_be_binding_of
	 <class 'rdflib.term.URIRef'> http://data.doremus.org/ontology#CLU198_should_have_container
	 <class 'rdflib.term.URIRef'> http://data.doremus.org/ontology#CLU206_should_have_media_type


In [16]:
# Récupérer les informations sur l'ontologie.
# Get information on the ontology.
infos = {}

for s, p, o in g.triples((rdflib.term.URIRef(being_imported),None, None)):
    infos[str(p)] = str(o)

    
# Distribuer ces informations dans des variables.
# En cas de problème, activer le print() en bas de la cellule
# et modifier les clés ici en fonction.
# Distribute these informations into variables.
# In case of trouble, activate the print() on the last line in the cell,
# and modify keys here according to the output.
nsuri = infos['http://purl.org/vocab/vann/preferredNamespaceUri']
ttldate = infos['http://purl.org/dc/terms/modified']
desctxt = infos['http://purl.org/dc/elements/1.1/description']
prefix = infos['http://purl.org/vocab/vann/preferredNamespacePrefix']

# Output test.
print(f"'nsuri' = {nsuri}")
print(f"'ttldate' = {ttldate}")
print(f"'desctxt' = {desctxt}")
print(f"'prefix' = {prefix}")

# En cas de problème :
# In case of trouble:
# print(infos)

'nsuri' = http://data.doremus.org/ontology#
'ttldate' = 2018-01-17
'desctxt' = DOREMUS is an extension of the FRBRoo model for describing the music.
'prefix' = mus


## Création du XML à produire

Création du XML à produire en vue de l'importation dans OntoME
> EN: *Creating the product XML to be imported into OntoME.*

In [17]:
### Créer l'élément racine: namespace
### Create the <namespace> root element.
namespace = et.Element("namespace")
namespace.tag, type(namespace)

('namespace', lxml.etree._Element)

In [18]:
### Ajouter l'élément standardLabel
### Add the <standardLabel> element.
standardLabel = et.SubElement(namespace, 'standardLabel', lang = 'en')
standardLabel.text = import_desc
print(et.tostring(namespace, pretty_print=True).decode('utf-8'))

<namespace>
  <standardLabel lang="en">DoReMus v0.2</standardLabel>
</namespace>



In [19]:
### Ajouter l'élément version
### Add the version element
version = et.SubElement(namespace, 'version')
version.text = import_version
print(et.tostring(namespace, pretty_print=True).decode('utf-8'))

<namespace>
  <standardLabel lang="en">DoReMus v0.2</standardLabel>
  <version>0.2 (05/06/2017)</version>
</namespace>



In [20]:
### Ajouter l'élément publishedAt
### Add the <publishedAt> element.

# Transformer la date au format xs:dateTime
# Transform date to xs:dateTime format.
anyletters = re.compile("[a-z]+")

if anyletters.search(ttldate) != None:
    evenmonths = {"April":"04", "June":"06", "September":"09", "November":"11"}
    oddmonths = {"January":"01", "March":"03", "May":"05", "July":"07", "August":"08", "October":"10", "December":"12"}

    year = re.search("\d{4}",crm.get('releaseDate')).group()
    month = re.search("[^\d\s]+",crm.get('releaseDate')).group()

    if month in evenmonths.keys():
        try:
            date = year + "-" + evenmonths[month] + "-30T23:59:59"
        except:
            print(f"It's an even month ({month}) but something's went wrong.")

    elif month in oddmonths.keys():
        try:
            date = year + "-" + oddmonths[month] + "-31T23:59:59"
        except:
            print(f"It's an odd month ({month}) but something's went wrong.")

    elif month == "February":
        try:
            if year in ["2016", "2020", "2024", "2028", "2032", "2036", "2040", "2044", "2048", "2052", "2056", "2060", "2064"]:
                date = year + "-02-29T23:59:59"
            else:
                date = year + "-02-28T23:59:59"
        except:
            print("It's February but something's went wrong.")
    else:
        print("I don't understand " + crm.get('releaseDate'))
else:
    date = ttldate + "T23:59:59"

### Créer et ajouter l'élément.
### Make and include element.
released = et.SubElement(namespace, 'publishedAt')
released.text = date
print(et.tostring(namespace, pretty_print=True).decode('utf-8'))

<namespace>
  <standardLabel lang="en">DoReMus v0.2</standardLabel>
  <version>0.2 (05/06/2017)</version>
  <publishedAt>2018-01-17T23:59:59</publishedAt>
</namespace>



In [21]:
### Ajouter les éléments racine de classes et properties
### Add the root elements for classes and properties.
classElement = et.Element('classes')
properties = et.Element('properties')
print(et.tostring(namespace, pretty_print=True).decode('utf-8'))

<namespace>
  <standardLabel lang="en">DoReMus v0.2</standardLabel>
  <version>0.2 (05/06/2017)</version>
  <publishedAt>2018-01-17T23:59:59</publishedAt>
  <classes/>
  <properties/>
</namespace>



In [22]:
### Deux méthodes d'inspection des enfants
### Two ways to inspect children.
namespace.getchildren(), [t for t in namespace]

([<Element standardLabel at 0x7f2b3ce5bc40>,
  <Element version at 0x7f2b3d241500>,
  <Element publishedAt at 0x7f2b3ce65180>,
  <Element classes at 0x7f2b3d276280>,
  <Element properties at 0x7f2b3d2a2880>],
 [<Element standardLabel at 0x7f2b3ce5bc40>,
  <Element version at 0x7f2b3d241500>,
  <Element publishedAt at 0x7f2b3ce65180>,
  <Element classes at 0x7f2b3d276280>,
  <Element properties at 0x7f2b3d2a2880>])

In [23]:
### Valider le XML produit jusqu'ici
# Normalement il proteste car les éléments <classes> and <properties> sont vides
### Validate the XML made so far.
# Python should protest because <classes> and <properties> elements are empty.
try:
    xmlschema.assert_(namespace)
except Exception as e:
    print(e)

Element 'classes': Missing child element(s). Expected is ( class ).


------------------------
## Déclarations de fonctions pour les classes et propriétés

> *Declaring functions for classes and properties.*

### FONCTION : Découper les URI...
...pour obtenir le nom de la classe/entité, son identifiant dans l'espace de nom, et l'espace de nom, séparés.

> *FUNCTION: cut URIs to obtain the name of the class/entity, its identifier in its own namespace and the namespace itself, apart from each other.*

EX :

| Variable | Value |
| -------: | :----- |
| about_val | 'http://iflastandards.info/ns/fr/frbr/frbroo/F24_Publication_Expression' |
| ns | `get_ns_nb('http://iflastandards.info/ns/fr/frbr/frbroo/')` -> '6' |
| cp | 'F24_Publication_Expression' |
| identifier | 'F24' |
| name | 'F24 Publication Expression' |

In [24]:
def cut_about(about_val):
    # Couper sur le # ou / -> espace de nom vs identifiant.
    # Cut on # or / sign -> namespace vs identifier
    
    try:
        if "#" in about_val:
            origns = about_val.split("#")[0] + "#"
            cp = about_val.split("#")[1]
        
        else:
            full = about_val.split("/")
            cp = full[-1]
            del full[-1]
            origns = "/".join(full)
        
        cpcut = cp.split("_")
        identifier = cpcut[0]
        del cpcut[0]
        name = "_".join(cpcut)
        
        if origns == nsuri:
            ns = "current"
            
        elif origns in ontome_ns.keys():
            ns = get_ns_nb(origns)
            
        else:
            ns = origns
        
        # print(ns, cp, identifier, name)
        return ns, cp, identifier, name
    
    
    except:
        print(about_val)

### FONCTION : récupérer les labels...

...avec leurs langues, et les retourner comme dictionnaire. Cette fonction utilise une requête SPARQL pour convoquer tous les objets d'un couple sujet-prédicat donné. Note : La requête SPARQL est composite car l'usage de variables cause une erreur (constaté dans la communauté rdflib).

> *FUNCTION: get the labels with their languages and return them as dictionaries. This function uses a SPARQL query to invoque all objects of a given subject-predicate couple. Note: The SPARQL query is composite, as using variables causes an error, as witnessed by the rdflib community.*

EX :

| Variable | Value |
| -------: | :----- |
| subject | 'http://data.doremus.org/ontology#M15_Dedication_Statement' |
| predicate | 'http://www.w3.org/2000/01/rdf-schema#label' |
| classDetails | `[{'lg':'en','txt':'Dedication Statement'}, {'lg':'fr','txt':'Mention de dédicace'}]` |

In [13]:
def get_languages(subject, predicate):

    classDetails = []
    links = " <" + str(subject) + "> <" + str(predicate) + "> "
    
    # Morceaux d'une requête SPARQL à compléter
    # avec les paramètres.
    # Bits of a SPARQL query, to be
    # completed by the parameters.
    begClass = """
    SELECT ?label ?language
    WHERE {"""
    endClass = """?label
        BIND ( lang(?label) AS ?language )
    }
    """
    
    # Construction des déclarations de préfixes
    # depuis le dictionnaire ns.
    # Constructing the prefix declaration
    # from the ns dictionary.
    prefixes = ""
    for prefix, link in ns.items():
        prefixes += f"PREFIX {prefix}: <{link}>\n"
    
    # Finalisation de la requête.
    # Finalisation of the query.
    fullClass = prefixes + begClass + links + endClass
    
    # Exécution de la requête.
    # Query execution.
    resultClass = g.query(fullClass)
    
    # Construction du retour depuis le résultat.
    # Constructing the return from the result.
    for row in resultClass:
        thisone = {'lg':row.language, 'txt':row.label}
        classDetails.append(thisone)
        
        
    return classDetails

### FONCTION : Transformer les labels...
...récupérés d'un identifiant, quand le label n'a pas été correctement renseigné.
> *FUNCTION: transform labels gotten from an identifier, when the label was incorrect.*

In [25]:
def transform_label(entry):
    
    label = str(entry)
    regex = re.compile(r"^[A-Z]+\d+i?(.*)$")
    
    # Si la str est un morceau d'URI.
    # If the str is a part of a URI.
    if "_" in label:
        splitlabel = label.split("_")
        if re.search("^[A-Z]+\d+i?.*$", splitlabel[0]) is not None:
            splitlabel.pop(0)
            return " ".join(splitlabel)
        else:
            print(f"What a strange label '{label}' is...")
    
    # Si l'identifiant est dedans.
    # If the identifier is inside.
    elif re.search("^[A-Z]+\d+i?.*$", label) is not None:
        splitlabel = label.split(" ")
        splitlabel.pop(0)
        return " ".join(splitlabel)
        
    else:
        return label

### FONCTION : déduire le numéro OntoME d'un NS...
... et ajouter l'identifiant OntoME de l'espace de nom dans le dictionnaire *ad hoc*.
> *FUNCTION: deduce the OntoME number of a namespace, and add it to the according dictionary.*

In [26]:
def get_ns_nb(link):
    
    # Parfois l'URI n'est pas renseignée pareil,
    # donc on essaiera tout.
    # Sometimes the URI is not entered the same,
    # so we are trying everything.
    link_l = [
        link,
        link + "/",
        link + "#"
    ]
    found = False
    
    for option in link_l:
        if found == False:
            
            if option in specified_versions.keys():
                version = specified_versions[option]
                nsref = ontome_ns[option]
                for ontome_version in nsref:
                    omvs = ontome_version['version']
                    answer = ontome_version['id']
                    if omvs == version:
                        ns_decl_list.append(str(answer))
                        return str(answer)
                        found = True
                        break
                        
            elif option in ontome_ns.keys():
                answer = ontome_ns[option][0]['id']
                ns_decl_list.append(str(answer))
                return str(answer)
                found = True
                break
                
    if found == False:
        print(link + " n'est pas renseigné dans le fichier des espaces de nom d'OntoME !")

-----------------------
## Ajouter les enfants des classes
> *Add the class children.*

In [27]:
### Reinitialiser le contenu de l'élément <classes>, puis la remplir.
### Clear the contents of the <classes> element, then fill it.

classElement.clear()
noEqCl = []
noUnion = []

for nsid in classes:
    
    # Déduire les informations nécessaires de l'URI.
    # Deduce the necessary information from the URI.
    class_ns, class_fullname, class_id, class_name = cut_about(str(nsid))
    
    # Vérifier que c'est bien une URI. Sinon, on ajoute un commentaire.
    # Check this is a URI. Otherwise we add a comment.
    if type(nsid) != rdflib.term.URIRef:
        classElement.append(et.Comment(str(nsid)))
        # Tentative avortée d'ajouter le nœud au commentaire.
        # Cancelled try to add the node into the comment.
        """ec = rdflib.Graph()
        for s, p, o in g.triples((nsid, None, None)):
            ec.add((s, p, o))
            for so, po, oo in g.triples((o, None, None)):
                ec.add((s, p, o))
        ttlcomment = ec.serialize(format='ttl')
        classElement.append(et.Comment(str(ttlcomment)))"""
    
    # Si c'est une URI, on détaille les propriétés.
    # If it is a URI, we detail properties.
    else:
        # Pour l'instant, on n'importe que les classes
        # propres au nouvel espace de nom.
        # For now, only import classes within the
        # new namespace.
        if being_imported in nsid:
            
            # Marquer la classe comme traitée.
            # Mark the class as dealt with.
            typed_decls[nsid] = "yes"
            
            # On récupère tous les labels.
            # Get all the labels.
            allLabels = get_languages(nsid, "http://www.w3.org/2000/01/rdf-schema#label")
            

            # On récupère tous les triplets de la classe.
            # On les mets dans le dictionnaire.
            # Get all triples with this class.
            # Put them in the dictionary.
            classDict = {}
            for s, p, o in g.triples((nsid, None, None)):
                if p in classDict:
                    classDict[p].append(o)
                else:
                    classDict[p] = [o]
                    
            # On commence à créer l'élément.
            # Underscore à cause du nom réservé.
            # Start making the element.
            # Underscore because the name is reserved.
            _class = et.SubElement(classElement,"_class")

            identifierInNamespace =  et.SubElement(_class,'identifierInNamespace')
            identifierInNamespace.text = class_id
            
            # Créer les éléments des labels.
            # Create the label elements.
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label') in classDict.keys():
                for label in allLabels:
                    standardLabel = et.Element('standardLabel')
                    standardLabel.set('lang', label['lg'])
                    standardLabel.text = transform_label(label['txt'])
                    _class.append(standardLabel)
                    del standardLabel
            else:
                standardLabel = et.SubElement(_class,'standardLabel', lang="en")
                standardLabel.text = transform_label(class_name)
            
            
            # Chercher une classe parente.
            # Look for a parent class.
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf') in classDict.keys():
                subClassOf_l = classDict[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf')]

                for scl in subClassOf_l:
                    scl_ns, scl_fullname, scl_id, scl_name = cut_about(str(scl))

                    subClassOf = et.SubElement(_class,'subClassOf')
                    subClassOf.text = scl_id
                    if being_imported not in scl:
                        nsnb = get_ns_nb(scl_ns)
                        subClassOf.set("referenceNamespace", nsnb)

            else:
                print(f"{class_fullname} n'est pas une sous-classe.")
            
            
            # Chercher les scopeNotes et exemples.
            # Get scopeNotes and examples.
            allscopes = []
            allex = []
            
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment') in classDict.keys():
                allnotes = get_languages(nsid, "http://www.w3.org/2000/01/rdf-schema#comment")
                for note in allnotes:
                    lowernote = note['txt'].lower()
                    if lowernote.startswith("ex"):
                        allex.append(note)
                    else:
                        allscopes.append(note)
            
            elif rdflib.term.URIRef("http://www.w3.org/2004/02/skos/core#scopeNote") in classDict.keys():
                allnotes = get_languages(nsid, "http://www.w3.org/2004/02/skos/core#scopeNote")
                
                for note in allnotes:
                    lowernote = note['txt'].lower()
                    if lowernote.startswith("ex"):
                        allex.append(note)
                    else:
                        allscopes.append(note)
            
            textProperties = et.Element('textProperties')
            
            if len(allscopes) != 0:
                for scope in allscopes:
                    scopeNote = et.Element('scopeNote', lang=scope['lg'])
                    scopeNote.text = scope['txt']
                    textProperties.append(scopeNote)
                    del scopeNote
            else:
                frScopeNote = et.SubElement(textProperties, 'scopeNote')
                frScopeNote.text = "Pas de scope note renseigné pour cette classe."
                frScopeNote.set('lang', 'fr')
                """
                enScopeNote = et.SubElement(textProperties, 'scopeNote')
                enScopeNote.text = "No scope note provided for this class."
                enScopeNote.set('lang', 'en')"""
                
            if len(allex) != 0:
                for ex in allex:
                    if ex['txt'].count('\n') > 1:
                        linelist = ex['txt'].split('\n')
                        for exline in linelist[1:]:
                            example = et.Element('example', lang=ex['lg'])
                            example.text = exline
                            textProperties.append(example)
                            del example
                    else:
                        example = et.Element('example', lang=ex['lg'])
                        example.text = ex['txt']
                        textProperties.append(example)
                        del example
            else:
                frexemple = et.SubElement(textProperties, 'example')
                frexemple.text = "Pas d'exemple renseigné pour cette classe."
                frexemple.set('lang', 'fr')
                """
                enexemple = et.SubElement(textProperties, 'example')
                enexemple.text = 'No example provided for this class.'
                enexemple.set('lang', 'en')"""
                
            _class.append(textProperties)
                
                
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#isDefinedBy') in classDict.keys():
                deftext = str(classDict[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#isDefinedBy')][0])
                if deftext != being_imported:
                    print(f"Il y a un 'rdfs:isDefinedBy' dans {class_id} à gérer.")

            _class.tag = "class"
            
            if rdflib.term.URIRef('http://www.w3.org/2002/07/owl#equivalentClass') in classDict.keys():
                print(f"owl:equivalentClass dans {class_fullname}")
                _class.append(et.Comment("owl:equivalentClass"))
                # Tentative de mettre le nœud dans le commentaire.
                """for element in classDict[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#equivalentClass')]:
                    ec = rdflib.Graph()
                    for s, p, o in g.triples((element, None, None)):
                        ec.add((s, p, o))
                    ttlcomment = ec.serialize(format='ttl')
                    _class.append(et.Comment("owl:equivalentClass\n" + str(ttlcomment)))"""

            else:
                noEqCl.append(class_fullname)
            
            if rdflib.term.URIRef('http://www.w3.org/2002/07/owl#unionOf') in classDict.keys():
                print(f"owl:unionOf dans {class_fullname}")
                _class.append(et.Comment("owl:unionOf"))
                # Tentative de mettre le nœud dans le commentaire.
                """for element in classDict[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#unionOf')]:
                    ec = rdflib.Graph()
                    for s, p, o in g.triples((element, None, None)):
                        ec.add((s, p, o))
                    ttlcomment = ec.serialize(format='ttl')
                    _class.append(et.Comment("owl:unionOf\n" + str(ttlcomment)))"""

            else:
                noUnion.append(class_fullname)
                    
# Si besoin, réactiver ces deux vérifications.
# print("Sans classe équivalente :\n\t- " + "\n\t- ".join(noEqCl) + "\n")
# print("Sans owl:UnionOf :\n\t- " + "\n\t- ".join(noUnion) + "\n")

owl:equivalentClass dans M4_Key


In [28]:
### Test output
print(et.tostring(namespace, pretty_print=True, encoding="utf-8").decode('utf-8'))

<namespace>
  <standardLabel lang="en">DoReMus v0.2</standardLabel>
  <version>0.2 (05/06/2017)</version>
  <publishedAt>2018-01-17T23:59:59</publishedAt>
  <classes>
    <!--ncf5996da2d484279af84d555871fa17cb1-->
    <!--ncf5996da2d484279af84d555871fa17cb4-->
    <class>
      <identifierInNamespace>M10</identifierInNamespace>
      <standardLabel lang="en">Catalogue Name</standardLabel>
      <standardLabel lang="fr">Nom de catalogue</standardLabel>
      <subClassOf referenceNamespace="1">E90</subClassOf>
      <textProperties>
        <scopeNote lang="fr">Pas de scope note renseigné pour cette classe.</scopeNote>
        <example lang="fr">Pas d'exemple renseigné pour cette classe.</example>
      </textProperties>
    </class>
    <class>
      <identifierInNamespace>M11</identifierInNamespace>
      <standardLabel lang="en">Catalogue Number</standardLabel>
      <standardLabel lang="fr">Numéro de catalogue</standardLabel>
      <subClassOf referenceNamespace="1">E90</subClassOf>


In [29]:
### Valider le document produit jusuq'ici
# Normalement il proteste car les éléments classes and properties sont vides.
# ou si l'extension/ontologie n'a pas renseigné assez d'infos sur ses classes.
try:
    xmlschema.assert_(namespace)
except Exception as e:
    print(e)

Element 'properties': Missing child element(s). Expected is ( property ).


----------------------
## Ajouter les enfants des propriétés

#### Test sur les cardinalités
Supprimé car aucune dans le premier import. À adapter depuis le script originel si besoin.

#### Récupérer les propriétés

In [1]:
### Reinitialiser le contenu de la balise 'properties',
#  puis la remplir
properties.clear()
erreurs = 0
invnb = 0
exportednb = 0

for nsid in props:
    
    # Déduire les informations nécessaires de l'URI en cas de problème.
    prop_ns, prop_fullname, prop_id, prop_name = cut_about(str(nsid))
    
    # Vérifier que c'est bien une URI. Sinon, on s'en occupera plus tard.
    if type(nsid) != rdflib.term.URIRef:
        propElement.append(et.Comment(str(nsid)))
        # Tentative de mettre le nœud dans le commentaire.
        """ec = rdflib.Graph()
        for s, p, o in g.triples((nsid, None, None)):
            ec.add((s, p, o))
            for so, po, oo in g.triples((o, None, None)):
                ec.add((s, p, o))
        ttlcomment = ec.serialize(format='ttl')
        propElement.append(et.Comment(str(ttlcomment)))"""
    
    # Si oui, on détaille les propriétés.
    else:
        
        # Vérifier que cette propriété n'est pas
        # une propriété inverse déclarée séparément.
        amIinv = re.search("^\w+\d+i\_.*$", prop_fullname)
        
        # Pour l'instant, on n'importe que les propriétés
        # propres au nouvel espace de nom.
        if being_imported in nsid and amIinv is None:
            
            typed_decls[nsid] = "yes"
            
            exportednb += 1
            
            # On récupère tous les labels.
            allLabels = get_languages(nsid, "http://www.w3.org/2000/01/rdf-schema#label")
            
            # On récupère tous les triplets de la propiété.
            # On les mets dans le dictionnaire.
            propDict = {}
            for s, p, o in g.triples((nsid, None, None)):
                if p in propDict:
                    propDict[p].append(o)
                else:
                    propDict[p] = [o]
            
            # On commence à créer l'élément.
            # underscore à cause du nom réservé
            _property = et.SubElement(properties,"_property")

            identifierInNamespace =  et.SubElement(_property,'identifierInNamespace')
            identifierInNamespace.text = prop_id
            
            
            
            
            # rdfs:labels and owl:inverseOf
            
            if rdflib.term.URIRef('http://www.w3.org/2002/07/owl#inverseOf') in propDict.keys():
                inverse_l = propDict[rdflib.term.URIRef('http://www.w3.org/2002/07/owl#inverseOf')]
                
                if len(inverse_l) == 1:
                    inverse = inverse_l[0]
                    typed_decls[inverse] = "yes"
                    invLabels = get_languages(inverse, "http://www.w3.org/2000/01/rdf-schema#label")
                    
                else:
                    _property.append(et.Comment(f"{prop_fullname} a plusieurs propriétés inverses !"))
                    invLabels = []
                    
                for s, p, o in g.triples((inverse, None, None)):
                    if p != rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label') and p != rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'):
                        _property.append(et.Comment(f"{str(p)} sur la propriété inverse."))
            else:
                invLabels = []
                    
            labelByLg = {}
            
            for labelentry in allLabels:
                lblg = str(labelentry['lg'])
                lbtxt = str(labelentry['txt'])
                if lblg in labelByLg.keys():
                    labelByLg[lblg]['label'] = lbtxt
                else:
                    labelByLg[lblg] = {'label':lbtxt}
                    
            for labelentry in invLabels:
                lblg = str(labelentry['lg'])
                lbtxt = str(labelentry['txt'])
                if lblg in labelByLg.keys():
                    labelByLg[lblg]['inv'] = lbtxt
                else:
                    labelByLg[lblg] = {'inv':lbtxt}
                       
            for language in labelByLg.keys():
                labelement = et.Element('label')
                labelement.set('lang', language)
                langdetail = labelByLg[language]
                
                if "label" in langdetail.keys():
                    standardLabel = et.Element('standardLabel')
                    standardLabel.text = transform_label(langdetail['label'])
                    labelement.append(standardLabel)
                    del standardLabel
                else:
                    standardLabel = et.Element('standardLabel')
                    standardLabel.text = transform_label(prop_name)
                    labelement.append(standardLabel)
                    del standardLabel
                        
                if "inv" in langdetail.keys():
                    inverseLabel = et.Element('inverseLabel')
                    inverseLabel.text = transform_label(langdetail['inv'])
                    labelement.append(inverseLabel)
                    del inverseLabel
                
                _property.append(labelement)
                del labelement
                
            # rdfs:subPropertyOf
            
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf') in propDict.keys():
                subPropOf_l = propDict[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf')]

                for spr in subPropOf_l:
                    spr_ns, spr_fullname, spr_id, spr_name = cut_about(str(spr))

                    subPropOf = et.SubElement(_property,'subPropertyOf')
                    
                    if spr == rdflib.term.URIRef("http://purl.org/dc/elements/1.1/title"):
                        print(f'{str(nsid)} a une classe ({str(spr)}) comme sur-propriété...')
                            
                    elif being_imported not in spr:
                        try:
                            subPropOf.text = spr_id
                            nsnb = get_ns_nb(spr_ns)
                            subPropOf.set("referenceNamespace", nsnb)
                            
                        except:
                            _property.append(et.Comment(f"rdfs:subPropertyOf {spr}."))
                            
                    else:
                        subPropOf.text = spr_id

            else:
                print(f"{prop_fullname} n'est pas une sous-propriété.")
                erreurs += 1
                
                
            
            # rdfs:domain
            
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain') in propDict.keys():
                domain_l = propDict[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain')]
                
                if len(domain_l) == 1:
                    domain = domain_l[0]
                    
                    if type(domain) == rdflib.term.URIRef:
                        dom_ns, dom_fullname, dom_id, dom_name = cut_about(str(domain))

                        hasDomain= et.Element('hasDomain')
                        hasDomain.text = dom_id

                        if being_imported not in domain:
                            nsnb = get_ns_nb(dom_ns)
                            hasDomain.set("referenceNamespace", nsnb)
                    else:
                        hasDomain= et.Element('hasDomain')
                        hasDomain.text = "rdfs:domain object is not an URI"
                        erreurs += 1
                else:
                    hasDomain= et.Element('hasDomain')
                    hasDomain.text = "Several"
                    erreurs += 1
                    
            else:
                hasDomain= et.Element('hasDomain')
                hasDomain.text = "None"
                erreurs += 1
            
            _property.append(hasDomain)
            
            # rdfs:hasRange
                
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range') in propDict.keys():
                range_l = propDict[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range')]
                
                if len(range_l) == 1:
                    prange = range_l[0]
                    
                    if type(prange) == rdflib.term.URIRef:

                        rn_ns, rn_fullname, rn_id, rn_name = cut_about(str(prange))

                        if prange == rdflib.term.URIRef("http://www.w3.org/2001/XMLSchema#string"):
                            hasRange= et.Element('hasRange')
                            hasRange.text = "E62"
                            hasRange.set('referenceNamespace', '1')
                            _property.append(hasRange)
                            del hasRange
                            
                        elif prange == rdflib.term.URIRef("http://www.w3.org/2001/XMLSchema#positiveInteger") or prange == rdflib.term.URIRef("http://www.w3.org/2001/XMLSchema#integer"):
                            hasRange= et.Element('hasRange')
                            hasRange.text = "E60"
                            hasRange.set('referenceNamespace', '1')
                            _property.append(hasRange)
                            del hasRange
                        
                        elif being_imported not in prange:
                            try:
                                hasRange= et.Element('hasRange')
                                nsnb = get_ns_nb(rn_ns)
                                hasRange.text = rn_id
                                hasRange.set("referenceNamespace", nsnb)
                                _property.append(hasRange)
                                del hasRange
                            except:
                                _property.append(et.Comment(f"rdfs:hasRange is {prange}"))
                                
                        else:
                            hasRange= et.Element('hasRange')
                            hasRange.text = rn_id
                            _property.append(hasRange)
                            del hasRange
                                
                    else:
                        hasRange = et.Element('hasRange')
                        hasRange.text = "rdfs:Range is not a URI"
                        _property.append(hasRange)
                        erreurs += 1
                else:
                    hasRange = et.Element('hasRange')
                    hasRange.text = "Several"
                    _property.append(hasRange)
                    erreurs += 1
                    
            else:
                hasRange = et.Element('hasRange')
                hasRange.text = "None"
                _property.append(hasRange)
                erreurs += 1
                
                
            
            # rdfs:comment ou skos:scopeNote
            
            # Scopenotes et exemples.
            
            allscopes = []
            allex = []
            
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#comment') in propDict.keys():
                allnotes = get_languages(nsid, "http://www.w3.org/2000/01/rdf-schema#comment")
                for note in allnotes:
                    lowernote = note['txt'].lower()
                    if lowernote.startswith("ex"):
                        allex.append(note)
                    else:
                        allscopes.append(note)

            elif rdflib.term.URIRef("http://www.w3.org/2004/02/skos/core#scopeNote") in propDict.keys():
                allnotes = get_languages(nsid, "http://www.w3.org/2004/02/skos/core#scopeNote")
                
                for note in allnotes:
                    lowernote = note['txt'].lower()
                    if lowernote.startswith("ex"):
                        allex.append(note)
                    else:
                        allscopes.append(note)
            
            textProperties = et.Element('textProperties')
            
            if len(allscopes) != 0:
                for scope in allscopes:
                    scopeNote = et.Element('scopeNote', lang=scope['lg'])
                    scopeNote.text = scope['txt']
                    textProperties.append(scopeNote)
                    del scopeNote
                
            if len(allex) != 0:
                for ex in allex:
                    if ex['txt'].count('\n') > 1:
                        linelist = ex['txt'].split('\n')
                        for exline in linelist[1:]:
                            example = et.Element('example', lang=ex['lg'])
                            example.text = exline
                            textProperties.append(example)
                            del example
                    else:
                        example = et.Element('example', lang=ex['lg'])
                        example.text = ex['txt']
                        textProperties.append(example)
                        del example
            
            _property.append(textProperties)
            
            # rdfs:isDefinedBy
            
            if rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#isDefinedBy') in propDict.keys():
                definition = propDict[rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#isDefinedBy')][0]
                if being_imported not in definition:
                    _property.append(et.Comment("rdfs:isDefinedBy\n" + str(definition)))
                    erreurs += 1
                    
                
            _property.tag = "property"
        
        elif amIinv is not None:
            invnb += 1
            
### Test output
print(f"\nIl y a {erreurs} erreurs mises en commentaire.\n")
print(f"\n{exportednb} propriétés ont été exportées.")
print(f"\n{invnb} propriétés étaient inverses.")
print(et.tostring(properties, pretty_print=True, encoding="utf-8").decode('utf-8'))

NameError: name 'properties' is not defined

## Assembler le XML final

In [ ]:
# Déclaration des espaces de noms en tête de document.
# Namespace declarations at the head of the document.
for ns_found in np.unique(ns_decl_list):
    current_ns = et.Element('referenceNamespace')
    current_ns.text = ns_found
    namespace.append(current_ns)
    del current_ns

# Ajout des éléments <classes> et <properties>.
# Adding the <classes> and <properties> elements.
namespace.append(classElement)
namespace.append(properties)

In [26]:
# Dernière vérification.
# Last check.

print(et.tostring(namespace, pretty_print=True).decode('utf-8'))

<namespace>
  <standardLabel lang="en">DoReMus v0.2</standardLabel>
  <version>0.2 (05/06/2017)</version>
  <publishedAt>2018-01-17T23:59:59</publishedAt>
  <classes>
    <!--n97b209e138e5463891588f1dc76e763ab1-->
    <!--n97b209e138e5463891588f1dc76e763ab4-->
    <class>
      <identifierInNamespace>M10</identifierInNamespace>
      <standardLabel lang="en">Catalogue Name</standardLabel>
      <standardLabel lang="fr">Nom de catalogue</standardLabel>
      <subClassOf referenceNamespace="1">E90</subClassOf>
      <textProperties>
        <scopeNote lang="fr">Pas de scope note renseign&#233; pour cette classe.</scopeNote>
        <example lang="fr">Pas d'exemple renseign&#233; pour cette classe.</example>
      </textProperties>
    </class>
    <class>
      <identifierInNamespace>M11</identifierInNamespace>
      <standardLabel lang="en">Catalogue Number</standardLabel>
      <standardLabel lang="fr">Num&#233;ro de catalogue</standardLabel>
      <subClassOf referenceNamespace="1">E9

In [27]:
### Valider le document produit jusqu'ici
try:
    xmlschema.assert_(namespace)
except Exception as e:
    print(e)

Element 'scopeNote', attribute 'lang': '' is not a valid value of the atomic type 'xs:language'.


---------------------
## Créer les outputs.

In [28]:
### Ecrire le document
dt = datetime.now()
tmsp = dt.strftime("%Y%m%d_%H%M%S")
# tmsp = ''
filename = f'output/output_{current_version}_{tmsp}.xml'
### Préparer l'arbre XML et l'écrire dans un fichier
# la méthode write() est disponible pour le type _ElementTree non pour _Element
tree = namespace.getroottree()
### xml_declaration=True, encoding="utf-8"
tree.write(filename, pretty_print=True, xml_declaration=True, encoding="utf-8")

In [ ]:
### Vérifier que tous les éléments avec un typage ont bien été traités par le script.
for identifier, used in typed_decls.items():
    if used == "no":
        print(identifier)

In [32]:
### Normalement cette syntaxe doit tester si le document de sortie est bien formé
with open(filename, 'r') as f:
    # txt = f.read()
    ### omet la xml_declaration qui ne peut pas être lue par et.fromstring()
    txt = ''.join(f.readlines()[1:])
    
    
try:
    # test_xmlf = et.parse(filename)
    test_xmlf = et.fromstring(txt)
    print(type(test_xmlf))
except Exception as e:
    print('Error: ' + str(e))

<class 'lxml.etree._Element'>


In [ ]:
print(et.tostring(test_xmlf, pretty_print=True, encoding="utf-8").decode('utf-8'))